In [ ]:
import tweepy
from tweepy import OAuthHandler
import json
from tweepy import Stream
from tweepy.streaming import StreamListener
import dataset  #per guardar els fichers en una BD
import time,threading  #l'utilitzare per fer que cada 10 segons s'executi el mirar si hi han nous tweets

In [ ]:
consumer_key = 'Jb1qld98VNlNX64rQYtre4dU8'  # tot aixo es necesari per connectarnos a la conta de twitter d'Alvaro
consumer_secret = 'HV9KBVHweaYKWn8F7kF7Nuhw3zqDI3Bot4sqtqFo6dxmqX7tY4'
access_token = '861315124459704320-RbiGm35PaghRHCLuZPu9u2IyP4GxaP2'
access_secret = 'lmL5IKKn8CtI5bkTahXC4ZkLKcfirHpjVsP2A88ZQWC3F'

CONNECTION_STRING = "sqlite:///tweets_hack.db"  #connexio a la base de dades, si esta buida crea un fitxer nou

auth = OAuthHandler(consumer_key, consumer_secret)  #per permetre accedir a Twitter desde Python
auth.set_access_token(access_token, access_secret)  #per permetre accedir a Twitter desde Python

api = tweepy.API(auth)                              #per permetre accedir a Twitter desde Python

In [ ]:
db = dataset.connect(CONNECTION_STRING)  #ens connectem a a Base de Dades

class MyListener(StreamListener):  #lo que llegeix tweets en LIVE
 
    def on_status(self, status):  # a la que rep un nou tweet entra aqui
        if status.retweeted_status:  #si es un retweet no el guardem ja que guardariem mes d'un cop el mateix missatge
            return
        ## ESPAI PER A DETERMINAR SI ES AVERIA O QUE, HA DE SER PROCESSAT SENCILL PERQUE SI TARDA MOLT CONNEXIO ES TENCA
        ## Si es averia, pregunta o queixa estaria bé classificar-ho aqui i aixi poder guardar en tweets_hack tambe quin dels 3 tipus es
        ## si es averia (if averia==1) en aquesta linea s'hauria d'enviar algo rapid a algú
        table = db["tweets_hack"]  #lo que afagirem a la base de dades
        try:
            table.insert(dict(
                user_location=status.user.location,  #localitzacio
                coordinates=status.coordinates,  #coordenades
                text=status.text,  #text
                user_name=status.user.screen_name,  #nom usuari
            )) 
        except ProgrammingError as err:
            print(err)
 
 
    def on_error(self, status):
        print(status)
        return True 
 
twitter_stream = Stream(auth, MyListener())  #estableix una sessio d'streaming
twitter_stream.filter(track=['@HackH2O_BCN'], async=True)  # es guarden tots els tweets en el ficher hackathon.json . async= True crec que el fa correr en un nou Thread

Webs mirades:
https://www.dataquest.io/blog/streaming-data-python/
https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/